In [ ]:
#| default_exp models.hint

# HINT

In [ ]:
#| export
import numpy as np
import torch
import torch.nn as nn

## 1. Reconciliation methods

In [ ]:
#| export
def get_bottomup_P(S: np.ndarray):
    n_series = len(S)
    n_agg = n_series-S.shape[1]
    P = np.zeros_like(S)
    P[n_agg:,:] = S[n_agg:,:]
    P = P.T
    return P

def get_mintrace_ols_P(S: np.ndarray):
    n_hiers, n_bottom = S.shape
    n_agg = n_hiers - n_bottom

    W = np.eye(n_hiers)

    # We compute reconciliation matrix with
    # Equation 10 from https://robjhyndman.com/papers/MinT.pdf
    A = S[:n_agg,:]
    U = np.hstack((np.eye(n_agg), -A)).T
    J = np.hstack((np.zeros((n_bottom,n_agg)), np.eye(n_bottom)))
    P = J - (J @ W @ U) @ np.linalg.pinv(U.T @ W @ U) @ U.T
    return P

def get_mintrace_wls_P(S: np.ndarray):
    n_hiers, n_bottom = S.shape
    n_agg = n_hiers - n_bottom
    
    W = np.diag(S @ np.ones((n_bottom,)))

    # We compute reconciliation matrix with
    # Equation 10 from https://robjhyndman.com/papers/MinT.pdf
    A = S[:n_agg,:]
    U = np.hstack((np.eye(n_agg), -A)).T
    J = np.hstack((np.zeros((n_bottom,n_agg)), np.eye(n_bottom)))
    P = J - (J @ W @ U) @ np.linalg.pinv(U.T @ W @ U) @ U.T
    return P

## 2. HINT

In [ ]:
#| export
class HINT:
    def __init__(self,
                 h: int,
                 model: nn.Module,
                 S: np.ndarray,
                 reconciliation: str):
        
        if model.h != h:
            raise Exception(f"Model h {model.h} does not match HINT h {h}")
        self.h = h
        self.model = model
        self.S = S
        self.reconciliation = reconciliation
        self.loss = model.loss

        available_reconcitliations = dict(
                                BottomUp=get_bottomup_P,
                                MinTraceOLS=get_mintrace_ols_P,
                                MinTraceWLS=get_mintrace_wls_P   
                                )

        if reconciliation not in available_reconcitliations:
            raise Exception(f"Reconciliation {reconciliation} not available")

        # Get SP matrix
        P = available_reconcitliations[reconciliation](S=S)
        self.SP = S @ P

        qs = torch.Tensor((np.arange(self.loss.num_samples)/self.loss.num_samples))
        self.sample_quantiles = torch.nn.Parameter(qs, requires_grad=False)


    def fit(self, dataset, val_size=0, test_size=0, random_seed=None):
        """ HINT.fit

        **Parameters:**<br>
        `dataset`: NeuralForecast's `TimeSeriesDataset` see details [here](https://nixtla.github.io/neuralforecast/tsdataset.html)<br>
        `val_size`: int, size of the validation set, (default 0).<br>
        `test_size`: int, size of the test set, (default 0).<br>
        `random_seed`: int, random seed for the prediction.<br>

        **Returns:**<br>
        `y_hat`: numpy predictions of the `NeuralForecast` model.<br>
        """
        self.model.fit(dataset=dataset,
                       val_size=val_size,
                       test_size=test_size,
                       random_seed=random_seed)

    def predict(self, dataset, step_size=1, random_seed=None, **data_module_kwargs):
        """ HINT.predict

        **Parameters:**<br>
        `dataset`: NeuralForecast's `TimeSeriesDataset` see details [here](https://nixtla.github.io/neuralforecast/tsdataset.html)<br>
        `step_size`: int, steps between sequential predictions, (default 1).<br>
        `random_seed`: int, random seed for the prediction.<br>
        `**data_kwarg`: additional parameters for the dataset module.<br>

        **Returns:**<br>
        `y_hat`: numpy predictions of the `NeuralForecast` model.<br>
        """
        num_samples = self.model.loss.num_samples

        # Hack to get samples by simulating quantiles (samples will be ordered)
        quantiles_old = self.model.loss.quantiles
        names_old = self.model.loss.output_names
        self.model.loss.quantiles = self.sample_quantiles
        self.model.loss.output_names = ['1']*num_samples
        samples = self.model.predict(dataset=dataset, 
                                       step_size=step_size,
                                       random_seed=random_seed,
                                       **data_module_kwargs)
        self.model.loss.quantiles = quantiles_old
        self.model.loss.output_names = names_old

        # Hack requires to break quantiles correlations between samples
        idxs = np.random.choice(num_samples, size=samples.shape, replace=True)
        aux_col_idx = np.arange(len(samples))[:,None] * num_samples
        idxs = idxs + aux_col_idx

        samples = samples.flatten()[idxs]
        samples = samples.reshape(dataset.n_groups, -1, self.h, num_samples)

        samples = np.einsum('ij,jwhp->iwhp', self.SP, samples)

        forecasts = np.quantile(samples, self.model.loss.quantiles, axis=-1)

        forecasts = forecasts.transpose(1,2,3,0)
        forecasts = forecasts.reshape(-1, len(self.model.loss.quantiles))

        return forecasts

    def set_test_size(self, test_size):
        self.model.test_size = test_size

    def save(self, path):
        """ BaseWindows.save

        Save the fitted model to disk.

        **Parameters:**<br>
        `path`: str, path to save the model.<br>
        """
        self.model.trainer.save_checkpoint(path)

In [ ]:
#| eval: false
def sort_df_hier(Y_df, S):
    Y_df.unique_id = Y_df.unique_id.astype('category')
    Y_df.unique_id = Y_df.unique_id.cat.set_categories(S.index)
    Y_df = Y_df.sort_values(by=['unique_id', 'ds'])

    return Y_df

In [ ]:
#| eval: false
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, PMM, GMM, NBMM
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic

from datasetsforecast.hierarchical import HierarchicalData

# Load TourismSmall dataset
Y_df, S_df, tags = HierarchicalData.load('./data', 'TourismSmall')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

Y_df = sort_df_hier(Y_df, S_df)

# Base model
nhits = NHITS(h=4,
             input_size=12,
             loss=GMM(n_components=2, level=[80,90]),
             max_steps=1000,
             early_stop_patience_steps=2,
             val_check_steps=100,
             scaler_type='robust',
             learning_rate=1e-3)

# HINT
model = HINT(h=4, model=nhits, S=S_df.values, reconciliation='MinTraceOLS')

# Fit and Predict
fcst = NeuralForecast(models=[model], freq='Q')
forecasts = fcst.cross_validation(df=Y_df, val_size=4, n_windows=1)

In [ ]:
#| eval: false
# Plot quantile predictions
unique_id = 'total'
Y_plot_df = Y_df[Y_df.unique_id==unique_id]
plot_df = forecasts[forecasts.unique_id==unique_id]
plot_df = Y_plot_df.merge(plot_df, on=['ds', 'unique_id'], how='left')

In [ ]:
#| eval: false
# Plot quantile predictions
plt.plot(plot_df['ds'], plot_df['y_x'], c='black', label='True')
plt.plot(plot_df['ds'], plot_df['HINT-median'], c='blue', label='median')
plt.fill_between(x=plot_df['ds'][-4:],
                 y1=plot_df['HINT-lo-90'][-4:].values,
                 y2=plot_df['HINT-hi-90'][-4:].values,
                 alpha=0.4, label='level 90')
plt.legend()
plt.grid()
plt.plot()